# How to transform data into factors

Based on a conceptual understanding of key factor categories, their rationale and popular metrics, a key task is to identify new factors that may better capture the risks embodied by the return drivers laid out previously, or to find new ones. 

In either case, it will be important to compare the performance of innovative factors to that of known factors to identify incremental signal gains.

We create the dataset here and store it in our [data](../../data) folder to facilitate reuse in later chapters.

## Imports & Settings

In [ ]:
import warnings

warnings.filterwarnings("ignore")

In [ ]:
%matplotlib inline

from datetime import datetime

import matplotlib.pyplot as plt
import pandas as pd
import pandas_datareader.data as web
import seaborn as sns
import statsmodels.api as sm
from sklearn.preprocessing import scale

# replaces pyfinance.ols.PandasRollingOLS (no longer maintained)
from statsmodels.regression.rolling import RollingOLS

In [ ]:
import talib

In [ ]:
import numpy as np

In [ ]:
sns.set_style("whitegrid")
idx = pd.IndexSlice

In [ ]:
import yfinance as yf

## Get Data

The `assets.h5` store can be generated using the the notebook [create_datasets](../../data/create_datasets.ipynb) in the [data](../../data) directory in the root directory of this repo for instruction to download the following dataset.

We load the Quandl stock price datasets covering the US equity markets 2000-18 using `pd.IndexSlice` to perform a slice operation on the `pd.MultiIndex`, select the adjusted close price and unpivot the column to convert the DataFrame to wide format with tickers in the columns and timestamps in the rows:

Set data store location:

# Otros Settings iniciales

In [ ]:
normaliza = True  # normalizamos por volatilidad
neutraliza = False  # normalizado cross sectional media y vola

In [ ]:
DATA_STORE = "../data/assets.h5"

In [ ]:
START = 1990
END = 2025


# Carga el dataframe desde el archivo csv
df = pd.read_csv('C:/Users/Inma/Documents/mis notebooks/Machine-Learning-for-Algorithmic-Trading-Second-Edition-master/data/tickers_darwinex.csv')

columna = df.columns[0]

# Añade '.US' al final de cada valor en la columna
df[columna] = df[columna].apply(lambda x: x + '.US')

# Almacena los valores de la primera columna en una lista. Los tickers
lista = df.iloc[:, 0].tolist()



# Cargamos los sectores y otra info
df_sect = pd.read_csv('C:/Users/Inma/Documents/mis notebooks/Machine-Learning-for-Algorithmic-Trading-Second-Edition-master/data/us_equities_meta_data.csv')


In [ ]:
# df_sect['ticker'] = df_sect['ticker'].apply(lambda x: x + '.US')

In [ ]:
# df_sect = df_sect.set_index('ticker')

In [ ]:
# len(df_sect)

In [ ]:
# df_sect.tail()

In [ ]:
# df_sect.info()

In [ ]:
# len(lista)

In [ ]:
# df_sect['esta_en_lista'] = df_sect.index.isin(lista)
# num_tickers_en_lista = df_sect['esta_en_lista'].sum()

In [ ]:
# num_tickers_en_lista

In [ ]:
# etfs a considerar
princi = [
    "XLE.US",
    "XLB.US",
    "XLI.US",
    "XLK.US",
    "XLF.US",
    "XLP.US",
    "XLY.US",
    "XLV.US",
    "XLU.US",
    "IYR.US",
    "VOX.US",
    "SPY.US",
]
# princi=lista

In [ ]:
# with pd.HDFStore(DATA_STORE) as store:
#    tickers = (store['stooq/us/nyse/etfs/prices'])

# str(START)

In [ ]:
# tickers=tickers.sort_index()

In [ ]:
# tickers.info()

In [ ]:
# print(tickers.index.names)

with pd.HDFStore(DATA_STORE) as store:
    prices1 = (store['stooq/us/nyse/etfs/prices']
              .loc[idx[princi, :], :])

with pd.HDFStore(DATA_STORE) as store:
    prices2 = (store['stooq/us/nyse/stocks/prices']
              .loc[idx[princi, :], :])   
with pd.HDFStore(DATA_STORE) as store:
    prices3 = (store['stooq/us/nasdaq/stocks/prices']
              .loc[idx[princi, :], :])  

In [ ]:
# prices = pd.concat([prices1, prices2, prices3])

In [ ]:
# prices.info()

In [ ]:
"""ticker_list = ['XLE', 'XLB', 'XLI', 'XLK', 'XLF', 
               'XLP', 'XLY', 'XLV', 'XLU', 'IYR', 'VOX', 'SPY']"""
ticker_list = ["XLE", "XLB", "XLI", "XLK", "XLF", "XLP", "XLY", "XLV", "XLU", "SPY"]

# Here we use yf.download function
data = yf.download(
    # passes the ticker
    tickers=ticker_list,
    # used for access data[ticker]
    group_by="ticker",
)

In [ ]:
# apilamos tickers
data = data.stack(-2)
data

In [ ]:
data = data.swaplevel(0, 1)

In [ ]:
data = data.rename_axis(["ticker", "date"])

In [ ]:
# Reordenar y renombrar las columnas directamente
new_order = ["Open", "High", "Low", "Close", "Volume"]
new_names = ["open", "high", "low", "close", "volume"]

# Reordenar las columnas
prices = data[new_order]

# Renombrar las columnas
prices.columns = new_names

In [ ]:
prices = prices.sort_index()

In [ ]:
# Crear un nuevo DataFrame sin entradas duplicadas en el índice
prices = prices.loc[~prices.index.duplicated(keep="first")]

In [ ]:
prices.info()

In [ ]:
prices.info()

## Select 500 most-traded stocks

In [ ]:
# si queremos selecionar los 500 o no
# selec_500= True
selec_500 = False

In [ ]:
if selec_500 == True:
    dv = prices.close.mul(prices.volume)
    top500 = (
        dv.groupby(level="date")
        .rank(ascending=False)
        .unstack("ticker")
        .dropna(thresh=8 * 52, axis=1)
        .mean()
        .nsmallest(500)
    )
    to_drop = prices.index.unique("ticker").difference(top500.index)
    len(to_drop)
    prices = prices.drop(to_drop, level="ticker")

In [ ]:
print(prices.index.unique("ticker"))

In [ ]:
# Suponiendo que 'df' es tu DataFrame
num_ticker = prices.index.get_level_values("ticker").nunique()

print(f'El índice "ticker" tiene {num_ticker} elementos únicos.')

In [ ]:
prices.head()

## eliminamos spy

In [ ]:
prices.index

In [ ]:
# eliminamos spy
prices = prices.drop(index="SPY", level=0)

In [ ]:
prices = prices.sort_index(
    level=list(range(len(prices.index.names)))
)  # Sort all levels

In [ ]:
# guardamos los datos de ohlcv
with pd.HDFStore(DATA_STORE) as store:
    store.put("data_close", prices.sort_index())
    print(store.info())

In [ ]:
# hacemos unstack de close sólo
prices = prices.loc[idx[:, str(START) : str(END)], "close"].unstack("ticker")

In [ ]:
prices

In [ ]:
tiene_indices_duplicados = prices.index.duplicated().any()

In [ ]:
tiene_indices_duplicados

### Keep data with stock info

Remove `stocks` duplicates and align index names for later joining.

In [ ]:
"""stocks = stocks[~stocks.index.duplicated()]
stocks.index.name = 'ticker'"""

Get tickers with both price information and metdata

In [ ]:
"""shared = prices.columns.intersection(stocks.index)"""

In [ ]:
# stocks = stocks.loc[shared, :]
# stocks.info()

In [ ]:
# prices = prices.loc[:, shared]
# prices.info()

In [ ]:
# assert prices.shape[1] == stocks.shape[0]

## Create weekly return series

To reduce training time and experiment with strategies for longer time horizons, we convert the business-daily data to week-end frequency using the available adjusted close price:

In [ ]:
weekly_prices_real = prices.resample(
    "M"
).last()  # para que las betas de fama french esten alineadas
weekly_prices = prices.resample("W").last()
# weekly_prices = prices.resample('W-WED').last() #final en miercoles

To capture time series dynamics that reflect, for example, momentum patterns, we compute historical returns using the method `.pct_change(n_periods)`, that is, returns over various weekly periods as identified by lags.

We then convert the wide result back to long format with the `.stack()` method, use `.pipe()` to apply the `.clip()` method to the resulting `DataFrame`, and winsorize returns at the [1%, 99%] levels; that is, we cap outliers at these percentiles.

Finally, we normalize returns using the geometric average. After using `.swaplevel()` to change the order of the `MultiIndex` levels, we obtain compounded weekly returns for six periods ranging from 1 to 12 weeks:

In [ ]:
weekly_prices.info()

In [ ]:
outlier_cutoff = 0.01
data = pd.DataFrame()
lags = [1, 2, 3, 6, 12, 52]  # para semanas
for lag in lags:
    data[f"return_{lag}w"] = (
        weekly_prices.pct_change(lag)
        .stack()
        .pipe(
            lambda x: x.clip(
                lower=x.quantile(outlier_cutoff), upper=x.quantile(1 - outlier_cutoff)
            )
        )
        .add(1)
        .pow(1 / lag)
        .sub(1)
    )
data = data.swaplevel().dropna()
data.info()

In [ ]:
# para fama frech
return_1w_real = weekly_prices_real.pct_change()

In [ ]:
return_1w_real = return_1w_real.stack().swaplevel().dropna()
return_1w_real.head()

In [ ]:
return_1w_real.name = "return_1w"

In [ ]:
data.head()

## Drop stocks with less than 10 yrs of returns

In [ ]:
# min_obs = 12*10 #mensual
min_obs = 52 * 10  # semanal
nobs = data.groupby(level="ticker").size()
keep = nobs[nobs > min_obs].index

data = data.loc[idx[keep, :], :]
data.info()

In [ ]:
data.index

In [ ]:
data.describe()

In [ ]:
data2 = data.copy()

## Nomalizado de retornos

In [ ]:
def normalize_by_rolling_std(series):
    return series / series.rolling(52).std().shift(1)

In [ ]:
# normaliza=True

In [ ]:
if normaliza == True:
    lags = [1, 2, 3, 6, 12, 52]  # para semanas
    for lag in lags:
        data[f"return_{lag}w"] = data.groupby(level="ticker")[
            f"return_{lag}w"
        ].transform(normalize_by_rolling_std)

In [ ]:
lag

In [ ]:
# Función para neutralizar (normalizar) los retornos por cada fecha


def neutralize(group):
    return (group - group.mean()) / group.std()

In [ ]:
# neutraliza=True

In [ ]:
if neutraliza == True:
    lags = [1, 2, 3, 6, 12, 52]
    for lag in lags:
        data[f"return_{lag}w"] = data.groupby(level="date")[f"return_{lag}w"].transform(
            neutralize
        )

In [ ]:
data.tail(20)

In [ ]:
data2.tail(20)

#creamos diferencias de retornos con spy
df_SPY = data.loc['SPY.US']
new_df = pd.DataFrame()
for ticker in data.index.get_level_values(0).unique():
    if ticker != 'SPY.US':
        df_temp = data.loc[ticker] - df_SPY
        df_temp['ticker'] = ticker
        new_df = pd.concat([new_df, df_temp])

new_df.set_index('ticker', append=True, inplace=True)
new_df = new_df.reorder_levels(['ticker', 'date'])

In [ ]:
# new_df

## Si queremos cambiar el target a excess return


In [ ]:
# data=new_df # si queremos cambiar el target a excess retur

In [ ]:
data.index.get_level_values(0).unique()

In [ ]:
# cmap = sns.diverging_palette(10, 220, as_cmap=True)
sns.clustermap(data.corr("spearman"), annot=True, center=0, cmap="Blues");

We are left with 1,670 tickers.

In [ ]:
data.index.get_level_values("ticker").nunique()

## Rolling Factor Betas

We will introduce the Fama—French data to estimate the exposure of assets to common risk factors using linear regression in [Chapter 8, Time Series Models]([](../../08_time_series_models)).

The five Fama—French factors, namely market risk, size, value, operating profitability, and investment have been shown empirically to explain asset returns and are commonly used to assess the risk/return profile of portfolios. Hence, it is natural to include past factor exposures as financial features in models that aim to predict future returns.

We can access the historical factor returns using the `pandas-datareader` and estimate historical exposures using the `PandasRollingOLS` rolling linear regression functionality in the `pyfinance` library as follows:

Use Fama-French research factors to estimate the factor exposures of the stock in the dataset to the 5 factors market risk, size, value, operating profitability and investment.

In [ ]:
factors = ["Mkt-RF", "SMB", "HML", "RMW", "CMA"]
factor_data = web.DataReader(
    "F-F_Research_Data_5_Factors_2x3", "famafrench", start="2000"
)[0].drop("RF", axis=1)
factor_data.index = factor_data.index.to_timestamp()
factor_data = factor_data.resample("M").last().ffill().div(100)

factor_data.index.name = "date"
factor_data.info()

In [ ]:
factor_data.tail(20)

### importante metemos el retorno sin adaptar para la regresión

In [ ]:
data2["return_1w"].name

In [ ]:
factor_data = factor_data.join(return_1w_real).sort_index()
factor_data.info()

In [ ]:
T = 24  # 2 years
betas = factor_data.groupby(level="ticker", group_keys=False).apply(
    lambda x: RollingOLS(
        endog=x.return_1w,
        exog=sm.add_constant(x.drop("return_1w", axis=1)),
        window=min(T, x.shape[0] - 1),
    )
    .fit(params_only=True)
    .params
    # .drop('const', axis=1)
)

In [ ]:
betas.describe().join(betas.sum(1).describe().to_frame("total"))

In [ ]:
cmap = sns.diverging_palette(10, 220, as_cmap=True)
sns.clustermap(betas.corr(), annot=True, cmap=cmap, center=0);

In [ ]:
betas.info()

In [ ]:
betas.tail(20)

In [ ]:
# betasc=betas.copy()

In [ ]:
"""# Añade dos semanas a cada fecha en tu índice ya que no estan disponibles en el momento de consulta

# Obtén el nivel 'date' del índice
dates = betas.index.get_level_values('date')

# Añade dos semanas a cada fecha
new_dates = dates + pd.DateOffset(weeks=10)

# Crea un nuevo MultiIndex con las nuevas fechas
new_index = pd.MultiIndex.from_arrays([betas.index.get_level_values('ticker'), new_dates], names=['ticker', 'date'])

# Asigna el nuevo índice a tu DataFrame
betas.index = new_index
"""

In [ ]:
betas.loc["XLK", "2002"].head(10)

In [ ]:
# incorporamos los cambios en los datos de betas
# for columna in betas.columns:
#    betas[columna + '_diff'] = betas[columna].diff().replace(0, np.nan).ffill()

In [ ]:
# stop

In [ ]:
data = data.join(
    betas.groupby(level="ticker").shift(1)
)  # hacemos shift pq lo conoceremos un mes despues
data.info()

In [ ]:
data["const"].head(30)

### Impute mean for missing factor betas

In [ ]:
factors = ["const", "Mkt-RF", "SMB", "HML", "RMW", "CMA"]

In [ ]:
# Ensure the result has the correct index structure
data[factors] = (
    data.groupby("ticker")[factors]
    .apply(lambda x: x.ffill())
    .reset_index(level=0, drop=True)
)
data.info()

In [ ]:
# incorporamos los cambios en los datos de betas
for columna in factors:
    data[columna + "_diff"] = data[columna].diff().replace(0, np.nan).ffill()

## Momentum factors

We can use these results to compute momentum factors based on the difference between returns over longer periods and the most recent weekly return, as well as for the difference between 3 and 12 week returns as follows:

In [ ]:
for lag in [2, 3, 6, 12, 52]:
    data[f"momentum_{lag}"] = data[f"return_{lag}w"].sub(data.return_1w)
data[f"momentum_3_12"] = data[f"return_12w"].sub(data.return_3w)

## Date Indicators

In [ ]:
dates = data.index.get_level_values("date")
# data['year'] = dates.year
data["month"] = dates.month

In [ ]:
print(dates)

## Sector

In [ ]:
data.tail()

In [ ]:
data.info()

In [ ]:
data.index

In [ ]:
# machacamos sector con industry que tiene más detalle
# df_sect['sector']=df_sect['industry']

In [ ]:
# sector = data.index.get_level_values('ticker')
# sec=pd.factorize(sector)[0].astype(int)
# data['sector'] = sec
# metemos el sector

# data= data.join(df_sect['sector'])

In [ ]:
# Crear una Serie con el índice del DataFrame y los valores del nivel 'ticker'
ticker_series = pd.Series(data.index.get_level_values("ticker"), index=data.index)

# Usar esta Serie para llenar los valores NA
data["sector"] = ticker_series

In [ ]:
data["sector"]

In [ ]:
data["sector"]

In [ ]:
data["sector"].unique()

In [ ]:
data[data["sector"].isna()].index.get_level_values(0).unique()

In [ ]:
data.info()

## Lagged returns

To use lagged values as input variables or features associated with the current observations, we use the .shift() method to move historical returns up to the current period:

In [ ]:
for t in range(1, 7):
    data[f"return_1w_t-{t}"] = data.groupby(level="ticker").return_1w.shift(t)
data.info()

## Target: Holding Period Returns

Similarly, to compute returns for various holding periods, we use the normalized period returns computed previously and shift them back to align them with the current financial features

In [ ]:
for t in [1, 2, 3, 6, 12]:
    data[f"target_{t}w"] = data.groupby(level="ticker")[f"return_{t}w"].shift(-t)

In [ ]:
cols = [
    "target_1w",
    "target_2w",
    "target_3w",
    "return_1w",
    "return_2w",
    "return_3w",
    "return_1w_t-1",
    "return_1w_t-2",
    "return_1w_t-3",
]

data[cols].dropna().sort_index().head(10)

In [ ]:
data.info()

## Create age proxy

We use quintiles of IPO year as a proxy for company age.

In [ ]:
"""data = (data
        .join(pd.qcut(stocks.ipoyear, q=5, labels=list(range(1, 6)))
              .astype(float)
              .fillna(0)
              .astype(int)
              .to_frame('age')))
data.age = data.age.fillna(-1)"""

## Create dynamic size proxy

We use the marketcap information from the NASDAQ ticker info to create a size proxy.

In [ ]:
# stocks.info()

Market cap information is tied to currrent prices. We create an adjustment factor to have the values reflect lower historical prices for each individual stock:

In [ ]:
"""size_factor = (weekly_prices
               .loc[data.index.get_level_values('date').unique(),
                    data.index.get_level_values('ticker').unique()]
               .sort_index(ascending=False)
               .pct_change()
               .fillna(0)
               .add(1)
               .cumprod())
size_factor.info()"""

In [ ]:
"""msize = (size_factor
         .mul(stocks
              .loc[size_factor.columns, 'marketcap'])).dropna(axis=1, how='all')"""

### Create Size indicator as deciles per period

Compute size deciles per week:

In [ ]:
"""data['msize'] = (msize
                 .apply(lambda x: pd.qcut(x, q=10, labels=list(range(1, 11)))
                        .astype(int), axis=1)
                 .stack()
                 .swaplevel())
data.msize = data.msize.fillna(-1)"""

## Combine data

In [ ]:
"""data = data.join(stocks[['sector']])
data.sector = data.sector.fillna('Unknown')"""

In [ ]:
data.info()

## The Data: Recessions & Leading Indicators

We will use a small and simple dataset so we can focus on the workflow. We use the Federal Reserve’s Economic Data (FRED) service (see Chapter 2) to download the US recession dates as defined by the National Bureau of Economic Research. We also source four variables that are commonly used to predict the onset of a recession (Kelley 2019) and available via FRED, namely:

The long-term spread of the treasury yield curve, defined as the difference between the ten-year and the three-week Treasury yield.
The University of Michigan’s consumer sentiment indicator
The National Financial Conditions Index (NFCI), and
The NFCI nonfinancial leverage subindex.

### Download from FRED

In [ ]:
indicators = [
    "JHDUSRGDPBR",
    "T10Y3M",
    "BAMLC0A0CM",
    "BAMLH0A0HYM2",
    "BAMLHE00EHYIOAS",
    "UMCSENT",
    "UNRATE",
    "GDPC1",
    "DCOILWTICO",
    "CORESTICKM159SFRBATL",
    "USSLIND",
    "VIXCLS",
    "OVXCLS",
    "ICSA",
    "MARTSMPCSM44000USS",
    "RSXFS",
    "TREAST",
    "DGS1",
]
var_names = [
    "recession",
    "yield_curve",
    "corp_oas",
    "hy_oas",
    "eu_hy_oas",
    "sentiment",
    "empleo",
    "real_gdp",
    "oil",
    "inflacion",
    "leading",
    "vix",
    "vixoil",
    "weekjobclaims",
    "retail_sales_percent",
    "retail_sales",
    "us_asset_balance",
    "1y_yield",
]

In [ ]:
features = var_names[1:]
label = var_names[0]

In [ ]:
var_display = [
    "Recession",
    "Yield Curve",
    "corp_oas",
    "hy_oas",
    "eu_hy_oas",
    "Sentiment",
    "empleo",
    "real_gdp",
    "oil",
    "inflacion",
    "leading",
    "vix",
    "vixoil",
    "weekjobclaims",
    "retail_sales_percent",
    "retail_sales",
    "us_asset_balance",
    "1y_yield",
]
col_dict = dict(zip(var_names, var_display))

In [ ]:
data_fred = (
    web.DataReader(indicators, "fred", 1980, END + 1).resample("W").last().ffill()
)
data_fred.columns = var_names

In [ ]:
data_fred.tail()

We standardize the features so all have mean 0 standard deviation of 1:

In [ ]:
# data_fred.loc[:, features] = scale(data_fred.loc[:, features])

In [ ]:
data_fred.info()

In [ ]:
data_fred.index.name = "date"

In [ ]:
"""# Añade  semanas a cada fecha en tu índice ya que no estan disponibles en el momento de consulta

# Añade dos semanas a cada fecha en tu índice
data_fred.index = data_fred.index + pd.DateOffset(weeks=1)
"""

In [ ]:
data_fred.tail()

In [ ]:
# data_fred = data_fred.drop(data_fred.index[-1]) #eliminamos el último registro que es repetido

In [ ]:
data_fred.head()

In [ ]:
# incorporamos los cambios en los datos
for columna in data_fred.columns:
    data_fred[columna + "_diff"] = data_fred[columna].diff().replace(0, np.nan).ffill()
    data_fred[columna + "_chg"] = (
        data_fred[columna].pct_change().replace(0, np.nan).ffill()
    )

In [ ]:
# eliminamos algunas variables que tienen mucha dependencia del nivel historico
data_fred = data_fred.drop(["empleo", "us_asset_balance"], axis=1)

In [ ]:
# para hacer bbfill sólo hasta que encuentre un primer valor
for columna in data_fred.columns:
    # Verificar si la columna tiene NaN al inicio
    if data_fred[columna].isna().iloc[0]:
        # Obtiene el primer valor no NaN de la columna
        primer_valor = data_fred[columna].dropna().iloc[0]
        # Rellena los NaN iniciales con el primer valor no NaN
        data_fred[columna][: data_fred[columna].first_valid_index()] = primer_valor

In [ ]:
data_fred.head(5)

In [ ]:
# data_fred.index = data_fred.index.to_timestamp()
data_fred.index.name = "date"

In [ ]:
data = data.join(data_fred)

# Data final shape
data.info()

## Store data

We will use the data again in several later chapters, starting in [Chapter 7 on Linear Models](../../07_linear_models/README.md).

In [ ]:
with pd.HDFStore(DATA_STORE) as store:
    store.put("engineered_features", data.sort_index())
    store.put("data_raw", data2.sort_index())  # antes de normalizado de retornos
    print(store.info())